### Define key loading and plotting functions

In [1]:
import dash
import plotly.express as px
from ome_zarr.io import parse_url
from ome_zarr.reader import Reader
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import json
import glob2 as glob
from skimage.measure import regionprops
import itertools
import os
from scipy.interpolate import LinearNDInterpolator
import alphashape


def load_nucleus_dataset(filename):
    # global fin_points_prev, not_fin_points_prev, class_predictions_curr, df, curationPath, propPath

    propPath = dataRoot + filename + '_nucleus_props.csv'

    if os.path.isfile(propPath):
        df = pd.read_csv(propPath, index_col=0)
    else:
        raise Exception(
            f"Selected dataset( {filename} ) dataset has no nucleus data. Have you run extract_nucleus_stats?")

#     fin_nuclei = np.where(df["pec_fin_flag"] == 1)
#     df = df.iloc[fin_nuclei]

    # normalize gene expression levels
    colnames = df.columns
    list_raw = [item for item in colnames if "_cell_mean_nn" in item]
    gene_names = [item.replace("_cell_mean_nn", "") for item in list_raw]

    for g in gene_names:
        ind_list = [i for i in range(len(colnames)) if g in colnames[i]]
        for ind in ind_list:
            colname = colnames[ind]
            c_max = np.max(df[colname])
            df[colname] = df.loc[:, colname] / c_max

    return df





/Users/nick/miniforge3/envs/napari-env/lib/python3.10/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


## Get list of files

In [2]:
dataRoot = "/Users/nick/Dropbox (Cole Trapnell's Lab)/Nick/pecFin/HCR_Data/nucleus_props/"
figureRoot = "/Users/nick/Dropbox (Cole Trapnell's Lab)/Nick/pecFin/HCR_figures/"
if os.path.isdir(figureRoot) == False:
    os.makedirs(figureRoot)
    
# get list of filepaths
fileList = sorted(glob.glob(dataRoot + '*_nucleus_props.csv'))
pdNameList = []
for fn in range(len(fileList)):
    labelName = fileList[fn].replace(dataRoot, '', 1)
    labelName = labelName.replace('_nucleus_props.csv', '')
    pdNameList.append(labelName)

# compile dictionary of gene names that correspond to each dataset
gene_name_dict = {}
for f in range(len(fileList)):
    filename_temp = pdNameList[f]
    propPath = dataRoot + filename_temp + '_nucleus_props.csv'

    if os.path.isfile(propPath):
        df_temp = pd.read_csv(propPath, index_col=0)
    else:
        raise Exception(
            f"Selected dataset( {filename_temp} ) dataset has no nucleus data. Have you run extract_nucleus_stats?")

    # get list of genes that we can look at
    colnames_temp = df_temp.columns
    list_raw = [item for item in colnames_temp if "_cell_mean_nn" in item]
    gene_names_temp = [item.replace("_cell_mean_nn", "") for item in list_raw]

    gene_name_dict[filename_temp] = gene_names_temp

## Test that plotting is working

In [28]:
import math 
import shutil

df_ind = 8

fileName = fileList[df_ind]
imageName = pdNameList[df_ind]

# gene_name = gene_name_dict[imageName][df_ind]

df = load_nucleus_dataset(imageName)
df_fin = df.iloc[np.where(df["pec_fin_flag"]==2)]


################
# basic nucleus position plot

r_vec = np.sqrt(df_fin["X"].to_numpy()**2 + df_fin["Y"].to_numpy()**2 + df_fin["Z"].to_numpy()**2)

# make save directory
frame_dir = figureRoot + imageName + "_raw_nucleus_frames/"
if os.path.isdir(frame_dir)==True:
    shutil.rmtree(frame_dir)

os.makedirs(frame_dir)

angle_vec_raw = np.linspace(1.25*np.pi, 3.25*np.pi, 25)
angle_vec_flipped = angle_vec_raw[::-1]
angle_vec = np.concatenate([angle_vec_raw, angle_vec_flipped[:-1]])

for iter_i, a in enumerate(angle_vec):
    angle = a
    za = 0.3
    vec = np.asarray([math.cos(angle), math.sin(angle), za])
    vec = vec*2
    camera = dict(
        eye=dict(x=vec[0], y=vec[1], z=vec[2]))

    fig = px.scatter_3d(df_fin, x="X", y="Y", z="Z", opacity=0.5, color=r_vec, 
                        color_continuous_scale="ice",template="plotly_white")
    
    fig.update_layout(scene_camera=camera, scene_dragmode='orbit')
    
    fig.update_layout(scene = dict(
                    xaxis_title='',
                    yaxis_title='',
                    zaxis_title='',
                    xaxis = dict(showticklabels=False),
                    yaxis = dict(showticklabels=False),
                    zaxis = dict(showticklabels=False)))
    
    fig.update_layout(coloraxis_showscale=False)
    
#     fig.show()
    fig.write_image(frame_dir + imageName + "_" + "{:03d}".format(iter_i) + ".png")
#     time.sleep(0.25)

In [14]:
gene_name_dict[imageName]

['Fgf10a', 'Robo3', 'Prdm1a']

In [31]:
angle_vec = np.linspace(1.25*np.pi, 3.25*np.pi, 25)

# make save directory
gene_frame_dir = figureRoot + imageName + "_gene_expression/"
if os.path.isdir(gene_frame_dir)==False:
#     shutil.rmtree(gene_frame_dir)
    os.makedirs(gene_frame_dir)

gene_name_list = gene_name_dict[imageName]
mRNA_suffix = "_cell_mean_nn"
color_scale_list = ["ice", "inferno", "viridis"]
top = 0.8
bottom = 0.2

total_counter = 0
for g in range(len(gene_name_list)):
    gene_name = gene_name_list[g]

    col_name = gene_name + mRNA_suffix
    mRNA_col = df_fin[col_name].copy()
    
    for iter_i, a in enumerate(angle_vec):
        angle = a
        za = -0.2
        vec = np.asarray([math.cos(angle), math.sin(angle), za])
        vec = vec*2
        camera = dict(
            eye=dict(x=vec[0], y=vec[1], z=vec[2]))

        fig = px.scatter_3d(df_fin, x="X", y="Y", z="Z", opacity=1, color=mRNA_col, range_color=[bottom, top],
                            color_continuous_scale=color_scale_list[g], template="plotly_white")

        fig.update_layout(scene_camera=camera, scene_dragmode='orbit')
        
        fig.update_layout(title_text=gene_name, title_x=0.5)
        
        fig.update_layout(scene = dict(
                        xaxis_title='',
                        yaxis_title='',
                        zaxis_title='',
                        xaxis = dict(showticklabels=False),
                        yaxis = dict(showticklabels=False),
                        zaxis = dict(showticklabels=False)))

        fig.update_layout(coloraxis_showscale=False)

#         fig.show()
        fig.write_image(gene_frame_dir + imageName + "_mRNA_" + "{:03d}".format(total_counter) + ".png")
        total_counter += 1